# IMPORTAÇÕES

In [ ]:
import pandas as pd
import numpy as np

from copy import copy
from math import inf
from pandas.api.types import is_numeric_dtype
from typing import Any, List

# FUNÇÕES

In [ ]:
class Environment:
    def __init__(self, root, mode, source, year, month, file, extension):
        self.root = root            # Pasta raiz
        self.mode = mode            # Pasta referente ao processamento de dados ("raw" [não processado] vs "cooked" [processado])
        self.source = source        # Pasta referente à fonte de dados
        self.year = year            # Ano
        self.month = month          # Mês
        self.file = file            # Nome do arquivo (sem extensão)
        self.extension = extension  # Extensão do arquivo

    def get_file_path(self):
      """
      Retorna caminho do arquivo.
      :return: Caminho do arquivo.
      """
      file_name = f"{self.file.format(year=self.year, month=self.month)}{self.extension}"
      parts = [
          self.root,
          self.mode,
          self.source,
          self.year,
          file_name,
      ]
      return "/".join(parts)

    def clone(self):
      """
      Retorna uma cópia rasa (shallow copy) do objeto. Ou seja,
      as referências não serão clonadas; apenas o objeto em si.
      :return: Cópia rasa do objeto.
      """
      return copy(self)

def int_to_str_with_zeros(n: int, size: int) -> str:
    return str(n).zfill(size)

def perror(e: Exception):
  print(f"Handled error of type: {type(e).__name__}")
  print(f"Error message: {e}")

def print_list(lst: List[List[Any]]):
  for el in lst:
    print(el)

# SETUP DO AMBIENTE

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


# CONCATENAÇÃO

## DEFINIÇÃO DA ROTINA

In [ ]:
def main():
  # PARÂMETROS ###############
  BEG_MONTH = 1
  FIN_MONTH = 12
  LEN_MONTH = 2
  BEG_YEAR = 2023
  FIN_YEAR = 2025
  LEN_YEAR = 4

  # AMBIENTE
  FILE_FORMAT = "focos_mensal_br_{year}{month}"

  ROOT = "/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets"
  MODE_IN = "cooked"
  SRC = "TerraBrasilis"
  FILE_IN = f"{FILE_FORMAT}_{MODE_IN}"
  EXT = ".csv"

  FILE_OUT = FILE_FORMAT
  ##############################

  # Intervalo de datasets
  MONTHS = [x for x in range(BEG_MONTH, FIN_MONTH + 1)]
  YEARS = [x for x in range(BEG_YEAR, FIN_YEAR + 1)]
  included = list()
  df = None
  dfs = list()

  # Ambiente de entrada
  env_in = Environment(ROOT, MODE_IN, SRC, BEG_YEAR, BEG_MONTH, FILE_IN, EXT)

  # Ambiente de saída
  env_out = env_in.clone()
  env_out.file = FILE_OUT
  env_out.year = "merged"
  env_out.month = ""

  for year in YEARS:
    env_in.year = int_to_str_with_zeros(year, LEN_YEAR)

    for month in MONTHS:
      env_in.month = int_to_str_with_zeros(month, LEN_MONTH)

      # ESTRUTURAÇÃO
      try:
        df = pd.read_csv(env_in.get_file_path())
      except (FileNotFoundError, ValueError, TypeError) as e:
        perror(e)
        df = None
        continue

      dfs.append(df)
      included.append([year, month])

  # !!! Investigar a necessidade de drop_duplicates !!!
  df = pd.concat(dfs, axis=0).sort_values(by=["_estado", "_ano", "_mes"]).drop_duplicates() # Concatena preservando as colunas e juntando as linhas

  # REMOÇÃO DE COLUNAS DESNECESSÁRIAS
  REMOVING = ["que_focos_qtd"]
  df.drop(columns=REMOVING, inplace=True)

  # SALVAMENTO
  try:
    print(f"\n\nSalvando em: {env_out.get_file_path()}\n")
    df.to_csv(env_out.get_file_path(), index=False, encoding="utf-8")
  except (FileNotFoundError, ValueError, TypeError) as e:
    perror(e)

  return included

## EXECUÇÃO DA ROTINA

In [ ]:
# Executa rotina de concatenação e coleta datasets incluídos
list_included = main()

Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/TerraBrasilis/2025/focos_mensal_br_202505_cooked.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/TerraBrasilis/2025/focos_mensal_br_202506_cooked.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/TerraBrasilis/2025/focos_mensal_br_202507_cooked.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_25_1/datasets/cooked/TerraBrasilis/2025/focos_mensal_br_202508_cooked.csv'
Handled error of type: FileNotFoundError
Error message: [Errno 2] No such file or directory: '/content/drive/MyDrive/07_per_shared/projCDat_

# LOG DO RESULTADO

In [ ]:
print("Incluídos:")
print_list(list_included)

Incluídos:
[2023, 1]
[2023, 2]
[2023, 3]
[2023, 4]
[2023, 5]
[2023, 6]
[2023, 7]
[2023, 8]
[2023, 9]
[2023, 10]
[2023, 11]
[2023, 12]
[2024, 1]
[2024, 2]
[2024, 3]
[2024, 4]
[2024, 5]
[2024, 6]
[2024, 7]
[2024, 8]
[2024, 9]
[2024, 10]
[2024, 11]
[2024, 12]
[2025, 1]
[2025, 2]
[2025, 3]
[2025, 4]
